In [ ]:
import re 

from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders  import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA



from langchain_community.vectorstores import FAISS

### Langgraph docs loader


In [ ]:
from langchain_community.document_loaders  import PyPDFLoader


loader = PyPDFLoader("./data/PERMENPAN NOMOR 38 TAHUN 2017.pdf")
documents = loader.load()

In [ ]:
documents[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA DAN\nREFORMASI BIROKRASI REPUBLIK INDONESIA\nNOMOR 38 TAHUN 2017\nTENTANG\nSTANDAR KOMPETENSI JABATAN APARATUR SIPIL NEGARA\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nMENTERI PENDAYAGUNAAN APARATUR NEGARA\nDAN REFORMASI BIROKRASI REPUBLIK INDONESIA,\nMenimbang : bahwa untuk melaksanakan ketentuan Pasal 55 ayat (5),\nPasal 109 ayat (4) dan ayat (5) dan Pasal 166 ayat (2)\nPeraturan Pemerintah Nomor 11 Tahun 2017 tentang\nManajemen Pegawai Negeri Sipil, perlu menetapkan Peraturan\nMenteri Pendayagunaan Aparatur Negara dan Reformasi\nBirokrasi tentang Standar Kompetensi Jabatan Aparatur Sipil\nNegara;\nMengingat : 1. Undang-Undang Nomor 5 Tahun 2014 tentang Aparatur\nSipil Negara (Lembaran Negara Republik Indonesia Tahun\n2014 Nomor 6, Tambahan Lembaran

In [ ]:
import pprint

pprint.pp(documents[0].metadata)

{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '',
 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf',
 'total_pages': 108,
 'page': 0,
 'page_label': '1'}


### Preprocessing

In [ ]:
import re 
def preprocess(text): 
    # Menghilangkan footer nomor halaman seperti "- 5 -"
    text = re.sub(r'-\s*\d+\s*-', '', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'\n(?!\n)', ' ', text)
    text = re.sub(r' +', ' ', text)

    return text.strip().lower()

for doc in documents:
    doc.page_content = preprocess(doc.page_content) 



In [ ]:
documents[0:5]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200,  
    length_function=len
)



docs_split = text_splitter.split_documents(documents)

In [ ]:
docs_split[:10]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

### Configuration

In [ ]:
BASE_URL = "https://vllm.emyulabs.my.id/v1"
API_KEY = "sk-local-test"

client = OpenAI(base_url=BASE_URL,
                api_key=API_KEY, 
                timeout=30.0,
                max_retries=3)

### Embedding model

In [ ]:

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings(
    base_url=BASE_URL,
    api_key=API_KEY,
    model="Qwen/Qwen3-Embedding-0.6B" 
)


db = FAISS.from_documents(docs_split, embedding_model)

### Instruct model

In [ ]:
from langchain_openai import ChatOpenAI
llm_chat = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY, 
    model="Qwen/Qwen2.5-coder-1.5B-Instruct", 
    temperature=0.7,
    max_tokens=256,
    streaming=True,
    verbose=True,
)

### Retrieval - Generation

In [ ]:
retriever = db.as_retriever(
    search_type = "similarity", # option = mmr or similarity_with_threshold
    search_kwargs = {"k": 10 }
)

# retriever_mmr = db.as_retriever(
#     # Ganti search_type menjadi "mmr"
#     search_type="mmr",
#     search_kwargs={
#         "k": 10,           
#         "fetch_k": 20,   
#         "lambda_mult": 0.5 
#     }
# )

query = """Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?"""

retrieved_docs = retriever.get_relevant_documents(query)

print(f"Dokumen terambil dari k :{retrieved_docs}")

Dokumen terambil dari k :[Document(id='c8e464cc-583e-41e9-badb-f6e306fa1cce', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 73, 'page_label': '74'}, page_content='pemahaman yang sama atas instruksi yang diterima/diberikan 1.3.mampu melaksanakan kegiatan surat menyurat sesuai tata naskah organisasi. 4. orientasi pada hasil 1 bertanggung jawab untuk 1.1.menyelesaikan tugas dengan tuntas; dapat diandalkan;'), Document(id='561fbe75-2275-46a0-bba4-6b93a77f128c', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 45, 'page_label': '46'}, page_content='dan pemahaman lintas organisasi untuk secara efektif memfasilitasi kebutuhan kelompok yang lebih besar dengan cara-cara yang mengikuti standar objektif, transparan, profesional, sehingga tidak merugikan para pihak di lingkup pelayanan publ

In [ ]:
#Prompt
from langchain.prompts import PromptTemplate

# Template ini akan diisi dengan dokumen dari retriever dan pertanyaan dari pengguna (query)
prompt_template = """
Gunakan potongan-potongan konteks berikut untuk menjawab pertanyaan pengguna.
Jawablah secara ringkas dan jelas dalam Bahasa Indonesia.
Jika Anda tidak tahu jawabannya berdasarkan konteks yang diberikan, katakan saja bahwa Anda tidak dapat menemukan informasinya, jangan mencoba mengarang jawaban.

KONTEKS:
{context}

PERTANYAAN:
{question}

JAWABAN YANG MEMBANTU:
"""

CUSTOM_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
from langchain.chains import RetrievalQA
rag_chain = RetrievalQA.from_chain_type(
    llm = llm_chat,
    chain_type = "stuff", 
    retriever = retriever,
    chain_type_kwargs = {"prompt": CUSTOM_PROMPT},
    return_source_documents=True
)

In [ ]:
# Menjalankan query melalui RAG Chain
response = rag_chain.invoke(query)

print("\nPERTANYAAN:")
print(query)

print("\nJAWABAN YANG DIHASILKAN LLM:")
print(response['result'])

print("\nDOCUMENT:")
print(response['source_documents'])


retrieved_docs


PERTANYAAN:
Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?

JAWABAN YANG DIHASILKAN LLM:
Peraturan Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi Tentang Standar Kompetensi Jabatan Aparatur Sipil Negara adalah pernyataan yang melibatkan beberapa aspek:

1. **Pendidikan dan Pengembangan Kompetensi**: Peraturan ini menunjukkan prioritas yang diberikan kepada pendidikan dan pengembangan kompetensi bagi calon calon jabatan sipil negara. Ini melibatkan proses pembentukan standar kompetensi yang harus dipenuhi oleh calon calon jabatan.

2. **Standar Kompetensi**: Standar kompetensi tersebut mencakup beberapa kompetensi penting seperti manajemen tim, komunikasi, kebijakan, dan manajemen proyek. Ini membantu agar calon calon jabatan memiliki kemampuan yang baik dalam mengimplementasikan tindakan dan strategi dalam pekerjaan.

3. **Pemberian Persetujuan**: Sebelum calon calon jab

[Document(id='c8e464cc-583e-41e9-badb-f6e306fa1cce', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 73, 'page_label': '74'}, page_content='pemahaman yang sama atas instruksi yang diterima/diberikan 1.3.mampu melaksanakan kegiatan surat menyurat sesuai tata naskah organisasi. 4. orientasi pada hasil 1 bertanggung jawab untuk 1.1.menyelesaikan tugas dengan tuntas; dapat diandalkan;'),
 Document(id='561fbe75-2275-46a0-bba4-6b93a77f128c', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 45, 'page_label': '46'}, page_content='dan pemahaman lintas organisasi untuk secara efektif memfasilitasi kebutuhan kelompok yang lebih besar dengan cara-cara yang mengikuti standar objektif, transparan, profesional, sehingga tidak merugikan para pihak di lingkup pelayanan publik unit kerjanya; 3.3. m

<!-- Penggunaan LLM sebagai Penilai  -->

### Penggunaan LLM untuk menilai skill / level competency 


In [ ]:
llm_assessor = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
    model="Qwen/Qwen2.5-coder-1.5B-Instruct",
    temperature=0.3,  # Lebih deterministik untuk penilaian
    max_tokens=512,
    streaming=False,
    verbose=False,
)

In [ ]:
from prompt.prompt import BASIC_RAG_PROMPT, MANAGERIAL_ASSESSMENT_PROMPT, TECHNICAL_COMPETENCY_PROMPT, COMPETENCY_GAP_ANALYSIS_PROMPT


In [ ]:
from prompt.prompt import MANAGERIAL_ASSESSMENT_PROMPT

def setup_rag_chain():
    """Setup RAG chain untuk query dasar"""
    return RetrievalQA.from_chain_type(
        llm=llm_chat,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": MANAGERIAL_ASSESSMENT_PROMPT},
        return_source_documents=True
    )

In [ ]:
def query_standar_kompetensi(pertanyaan):
    """
    Fungsi dasar untuk query standar kompetensi
    """
    rag_chain = setup_rag_chain()
    response = rag_chain.invoke(pertanyaan)
    
    return {
        'result': response['result'],
        'source_documents': response['source_documents']
    }

In [ ]:
def assess_competency(nama, jabatan, jawaban_test, kompetensi, level_target):
    """
    Menilai kompetensi berdasarkan jawaban test case
    """
    # Retrieve dokumen relevan
    query = f"kompetensi {kompetensi} level {level_target} indikator perilaku"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    # Setup chain untuk penilaian
    assessment_chain = RetrievalQA.from_chain_type(
        llm=llm_assessor,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": MANAGERIAL_ASSESSMENT_PROMPT},
        return_source_documents=False
    )
    
    # Eksekusi penilaian
    assessment_result = assessment_chain.invoke({
        "query": "",
        "context": context,
        "nama": nama,
        "jabatan": jabatan,
        "jawaban": jawaban_test,
        "kompetensi": kompetensi,
        "level_target": level_target
    })
    
    return assessment_result['result']

In [ ]:
def analyze_competency_gap(nama, jabatan, level_sekarang, level_target, hasil_assessment):
    """
    Menganalisis gap kompetensi dan memberikan rekomendasi pengembangan
    """
    query = f"kompetensi manajerial level {level_target} pengembangan"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    gap_analysis_chain = RetrievalQA.from_chain_type(
        llm=llm_assessor,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": COMPETENCY_GAP_ANALYSIS_PROMPT},
        return_source_documents=False
    )
    
    analysis_result = gap_analysis_chain.invoke({
        "query": "",
        "context": context,
        "nama": nama,
        "jabatan": jabatan,
        "level_sekarang": level_sekarang,
        "level_target": level_target,
        "hasil_assessment": hasil_assessment
    })
    
    return analysis_result['result']


In [ ]:
def assess_technical_competency(nama, jabatan, bidang_teknis, jawaban, level_target):
    """
    Menilai kompetensi teknis spesifik
    """
    query = f"kompetensi teknis {bidang_teknis} level {level_target}"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    technical_chain = RetrievalQA.from_chain_type(
        llm=llm_assessor,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": TECHNICAL_COMPETENCY_PROMPT},
        return_source_documents=False
    )
    
    result = technical_chain.invoke({
        "query": "",
        "context": context,
        "nama": nama,
        "jabatan": jabatan,
        "bidang_teknis": bidang_teknis,
        "jawaban": jawaban,
        "level_target": level_target
    })
    
    return result['result']

In [ ]:
def demo_basic_rag():
    """Demonstrasi fungsi RAG dasar"""
    print("\n" + "="*60)
    print("DEMO: BASIC RAG SYSTEM")
    print("="*60)
    
    questions = [
        "Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA DAN REFORMASI BIROKRASI TENTANG STANDAR KOMPETENSI JABATAN APARATUR SIPIL NEGARA?",
        "Apa saja jenis kompetensi menurut Permenpan RB 38/2017?",
        "Jelaskan tentang kompetensi manajerial level 3",
        "Apa yang dimaksud dengan sistem merit dalam standar kompetensi ASN?"
    ]
    
    for i, question in enumerate(questions, 1):
        print(f"\n{i}. PERTANYAAN: {question}")
        result = query_standar_kompetensi(question)
        print(f"JAWABAN: {result['result']}")
        print("-" * 50)


In [ ]:
def demo_competency_assessment():
    """Demonstrasi sistem penilaian kompetensi"""
    print("\n" + "="*60)
    print("DEMO: COMPETENCY ASSESSMENT SYSTEM")
    print("="*60)
    
    # Test Case 1: Kompetensi Manajerial
    test_case_1 = {
        "nama": "Budi Santoso",
        "jabatan": "Kepala Seksi",
        "jawaban_test": """
        Dalam menyelesaikan konflik tim, saya pertama-tama mengumpulkan semua pihak 
        untuk mendengarkan perspektif masing-masing. Saya menggunakan pendekatan win-win solution 
        dengan mencari titik temu yang menguntungkan semua pihak. Setelah mencapai kesepakatan, 
        saya memastikan implementasinya dengan monitoring berkala dan evaluasi hasil.
        Saya juga mendokumentasikan pembelajaran dari konflik ini untuk pencegahan di masa depan.
        """,
        "kompetensi": "manajerial",
        "level_target": 3
    }
    
    print("\n📊 TEST CASE 1: PENILAIAN KOMPETENSI MANAJERIAL LEVEL 3")
    print(f"Kandidat: {test_case_1['nama']} - {test_case_1['jabatan']}")
    
    # Melakukan penilaian
    hasil_penilaian = assess_competency(
        nama=test_case_1["nama"],
        jabatan=test_case_1["jabatan"],
        jawaban_test=test_case_1["jawaban_test"],
        kompetensi=test_case_1["kompetensi"],
        level_target=test_case_1["level_target"]
    )
    
    print("HASIL PENILAIAN:")
    print(hasil_penilaian)
    
    # Analisis gap
    print("\n🔍 ANALISIS GAP KOMPETENSI:")
    gap_analysis = analyze_competency_gap(
        nama=test_case_1["nama"],
        jabatan=test_case_1["jabatan"],
        level_sekarang=2,
        level_target=test_case_1["level_target"],
        hasil_assessment=hasil_penilaian
    )
    print(gap_analysis)
    
    # Test Case 2: Kompetensi Teknis
    test_case_2 = {
        "nama": "Siti Rahayu",
        "jabatan": "Analis Kepegawaian",
        "bidang_teknis": "manajemen SDM",
        "jawaban_test": """
        Untuk menyusun perencanaan SDM, saya melakukan analisis kebutuhan berdasarkan 
        rencana strategis organisasi. Saya menggunakan data historis dan proyeksi kebutuhan 
        untuk menentukan jumlah dan kualifikasi pegawai yang dibutuhkan. 
        Saya juga mempertimbangkan aspek pengembangan karir dan suksesi planning.
        """,
        "level_target": 2
    }
    
    print("\n" + "="*50)
    print("\n📊 TEST CASE 2: PENILAIAN KOMPETENSI TEKNIS")
    print(f"Kandidat: {test_case_2['nama']} - {test_case_2['jabatan']}")
    print(f"Bidang: {test_case_2['bidang_teknis']} - Level: {test_case_2['level_target']}")
    
    hasil_teknis = assess_technical_competency(
        nama=test_case_2["nama"],
        jabatan=test_case_2["jabatan"],
        bidang_teknis=test_case_2["bidang_teknis"],
        jawaban=test_case_2["jawaban_test"],
        level_target=test_case_2["level_target"]
    )
    
    print("HASIL PENILAIAN TEKNIS:")
    print(hasil_teknis)


In [ ]:
demo_basic_rag()

In [ ]:
demo_competency_assessment()

<!--  -->